<a href="https://colab.research.google.com/github/LidiiaMelnyk95/FSU_Jena_scripts/blob/main/text_classification_with_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
!pip uninstall tensorflow
!pip install tensorflow==2.8.0

In [ ]:
import pandas as pd

class_list = ['2','1','0']

df = pd.read_csv('/content/oversampled.csv', sep = ',')

df = df.drop(columns = 'Unnamed: 0')
df['Label'] = df['Label'].replace({'1':1, '2':2, '0':0})

In [ ]:
df.head()

,Text,Label
0,Und ich mag sie so wie sie auch wenn sie ein M...,2
1,@Fisch01 Also erstmal geht es in der Biologie ...,2
2,Ich lese mir grad diese Kommentare durch. Ach ...,2
3,Ich fände es interessant wenn man gefragt hätt...,2
4,"@CrucioDG Kraft, Atorität, Dominanz Durchsetzu...",2


In [ ]:

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (7819, 2)
test shape:  (1955, 2)


In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "distilbert-base-german-cased",
    num_labels=4,
    args=train_args,
    use_cuda = False,
)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.5.attention.out_lin.weight', 'distilbert.transformer.layer.1.ffn.lin1.weight', 'distilbert.transformer.layer.5.sa_layer_norm.weight', 'distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.3.attention.out_lin.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.3.ffn.lin2.weight', 'distilbert.transformer.layer.5.output_layer_norm.weight', 'distilbert.transformer.layer.2.output_layer_norm.bias', 'distilbert.transformer.layer.0.output_layer_norm.bias', 'distilbert.transformer.layer.4.attention.v_lin.weight', 'distilbert.transformer.layer.1.sa_layer_no

In [ ]:
model.train_model(train_df)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)
print(result, model_outputs, wrong_predictions)


In [ ]:
import os
import tarfile

def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

# run the function
pack_model('output_path','model_name')

In [ ]:
class_list = ['2','1','0y']
test_comment = "Ich kann diese Bewegung nicht verstehen"

predictions, raw_outputs = model.predict([test_comment])

print(class_list[predictions[0]])
# OTHER
test_comment2 = "ich finde das geht jetzt gar nicht"

predictions, raw_outputs = model.predict([test_comment2])

print(class_list[predictions[0]])